# МатМод ЛР3

Имитация систем непрерывных и дискретных случайных
величин

In [1]:
# Setting matplotlib backend:

%matplotlib notebook

# Imports:

import random
from typing import NamedTuple

import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d

import sympy as sp

from IPython.display import display as ipydisplay, Math, Latex

In [2]:
# Toggles & settings:

USE_CUSTOM_RANDOM_FUNCTION = False

if USE_CUSTOM_RANDOM_FUNCTION:
    # Source: https://en.wikipedia.org/wiki/Linear_congruential_generator#Parameters_in_common_use
    # Using glibc params
    lcg_X = 42   # seed
    def lcg():
        global lcg_X
        a = 1103515245
        c = 12345
        m = 2**31
        lcg_X = ((a*lcg_X + c) % m) / m
        return lcg_X
    rand = lcg
else:
    rand = random.random

## Задание 1

Написать программу, реализующую метод формирования двумерной НСВ с определенным распределением.

Выполнить статистическое исследование:
1. Проверить составляющие двумерной НСВ на независимость;
2. Найти условные плотности распределения;
3. Построить гистограммы составляющих двумерной НСВ и графики их плотностей распределения в одной системе координат;
4. Построить гистограмму распределения двумерной НСВ и график плотности распределения в одной системе координат (3D-график);
5. Вычислить теоретические, точечные и интервальные значения характеристик двумерной НСВ (математическое ожидание, дисперсия, корреляция);
6. *Проверить статистические гипотезы о соответствии полученных оценок характеристик (математическое ожидание, дисперсия, корреляция) случайной величины теоретическим.


In [3]:
# Исследуемый закон распределения:

# Вариант 7

# Плотность распределения:

f_str = '0.5 * cos(x-y)'
f_sp = sp.sympify(f_str)
f = sp.lambdify(sp.symbols('x, y'), f_sp)
x_sp, y_sp = sp.symbols('x y')

ipydisplay(Math(f'f(x, y) = {f_sp}'))

<IPython.core.display.Math object>

In [4]:
# Специальный тип для функции плотности:

class Pdf2dBundle(NamedTuple):
    pdf: callable
    x1: float
    x2: float
    y1: float
    y2: float

f_bundle = Pdf2dBundle(
    pdf=f,
    x1=0,
    x2=np.pi/2,
    y1=0,
    y2=np.pi/2,
)

In [5]:
# Функция распределения (CDF):

F_sp = sp.integrate(
    f_sp,
    (x_sp, f_bundle.x1, x_sp),
    (y_sp, f_bundle.y1, y_sp),
)
F = sp.lambdify(sp.symbols('x, y'), F_sp)

ipydisplay(Math(f'F(x, y) = {sp.latex(sp.simplify(F_sp))}'))

<IPython.core.display.Math object>

In [6]:
# 3D график плотности распределения

node_count = 16

x_list = np.linspace(
    f_bundle.x1,
    f_bundle.x2,
    node_count,
)
y_list = np.linspace(
    f_bundle.y1,
    f_bundle.y2,
    node_count,
)
X, Y = np.meshgrid(x_list, y_list)
Z = f(X, Y)

fig = plt.figure(label='Плотность распределения')
ax = plt.axes(projection='3d')
ax.plot_wireframe(X, Y, Z, color='green')

plt.show()

<IPython.core.display.Javascript object>

In [7]:
plt.clf()

In [8]:
# 3D график функции плотности распределения

Z = F(X, Y)

fig = plt.figure(label='Функция распределения')
ax = plt.axes(projection='3d')
ax.plot_wireframe(
    X, Y, Z,
    color='green',
)

plt.show()

<IPython.core.display.Javascript object>

In [9]:
plt.clf()

In [8]:
# Проверка, что значение определенного интеграла функции плотности распределения равно 1:

integral_value = sp.integrate(
    f_sp,
    (x_sp, f_bundle.x1, f_bundle.x2),
    (y_sp, f_bundle.y1, f_bundle.y2),
)

ipydisplay(Math(f'\\int_{{0}}^{{\\pi/2}} \\int_{{0}}^{{\\pi/2}} {f_str} \,dx dy = {round(float(integral_value), 2)}'))

<IPython.core.display.Math object>

### 1. Проверить составляющие двумерной НСВ на независимость

$(f(x, y) = f(x) * f(y) \rightarrow X, Y независимы)$

In [9]:
# Одномерные плотности:

f_x_sp = sp.integrate(
    f_sp,
    (y_sp, f_bundle.x1, f_bundle.x2)
)
f_x = sp.lambdify(
    sp.symbols('x'),
    f_x_sp,
)

f_y_sp = sp.integrate(
    f_sp,
    (x_sp, f_bundle.y1, f_bundle.y2)
)
f_y = sp.lambdify(
    sp.symbols('y'),
    f_y_sp,
)

ipydisplay(Math(f'f(x) = {sp.latex(sp.simplify(f_x_sp))}'))
ipydisplay(Math(f'f(y) = {sp.latex(sp.simplify(f_y_sp))}'))

multiplied_sp = sp.Mul(f_x_sp, f_y_sp)
multiplied = sp.lambdify(sp.symbols('x, y'), multiplied_sp)
ipydisplay(Math(f'f(x)*f(y) = {sp.latex(sp.simplify(multiplied_sp))}'))



<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Javascript object>

In [ ]:
# График разности f(x, y) и f(x)*f(y):

Z = f(X, Y) - multiplied(X, Y)
fig = plt.figure(label='Разность f(x, y) и f(x)*f(y)')
ax = plt.axes(projection='3d')
ax.plot_wireframe(X, Y, Z, color='green')

plt.show()

In [ ]:
plt.clf()

### 2. Найти условные плотности распределения

In [10]:
f_x_pipe_y_sp = f_sp / f_y_sp
f_y_pipe_x_sp = f_sp / f_x_sp

ipydisplay(Math(f'f(x | y) = {sp.latex(sp.simplify(f_x_pipe_y_sp))}'))
ipydisplay(Math(f'f(y | x) = {sp.latex(sp.simplify(f_y_pipe_x_sp))}'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### 3. Построить гистограммы составляющих двумерной НСВ и графики их плотностей распределения в одной системе координат

In [11]:
# Функция для генерации выборки по функции плотности:

sample_size = 1_000_000

def generate_sample(pdf_bundle: Pdf2dBundle, sample_size=sample_size):
    sample = []
    while len(sample) < sample_size:
        x = pdf_bundle.x1 + rand() * (pdf_bundle.x2 - pdf_bundle.x1)
        y = pdf_bundle.y1 + rand() * (pdf_bundle.y2 - pdf_bundle.y1)
        z = rand()
        if z < pdf_bundle.pdf(x, y):
            sample.append((x, y))
    return sample

In [12]:
# Построение выборки:

sample = generate_sample(f_bundle)
x_sample = [el[0] for el in sample]
y_sample = [el[1] for el in sample]

In [13]:
# Построение гистограммы для иксов:

fig = plt.figure(label='Гистограмма 1')

plt.hist(
    x_sample,
    density=True,
    bins=10,
)
plt.plot(
    x_list,
    f_x(x_list),
)
plt.show()

<IPython.core.display.Javascript object>

In [ ]:
plt.clf()

In [14]:
# Построение гистограммы для игреков:

fig = plt.figure(label='Гистограмма 2')

plt.hist(
    y_sample,
    density=True,
    bins=10,
)
plt.plot(
    y_list,
    f_y(y_list),
)
plt.show()

<IPython.core.display.Javascript object>

In [ ]:
plt.clf()

### 4. Построить гистограмму распределения двумерной НСВ и график плотности распределения в одной системе координат (3D-график)

In [22]:
# Двумерная гистограмма + график плотности распределения:

fig = plt.figure(label='Гистограмма 3')
ax = fig.add_subplot(111, projection='3d')

bin_count = 16

x_bin_width = (f_bundle.x2 - f_bundle.x1) / bin_count
y_bin_width = (f_bundle.y2 - f_bundle.y1) / bin_count

hist, x_edges, y_edges = np.histogram2d(
    x_sample, y_sample,
    bins=bin_count,
    range=[
        [f_bundle.x1, f_bundle.x2],
        [f_bundle.y1, f_bundle.y2],
    ],
)

x_pos, y_pos = np.meshgrid(
    x_edges[:-1],
    y_edges[:-1],
)

hist = hist / sample_size / x_bin_width / y_bin_width
heights = hist.flatten()

ax.bar3d(
    x_pos.flatten(), y_pos.flatten(), np.zeros(len(heights)),
    x_bin_width, y_bin_width, heights,
    alpha=0.1
)
Z = f(X, Y)
ax.plot_surface(
    X, Y, Z,
    color='green',
)

plt.show()

<IPython.core.display.Javascript object>

In [ ]:
plt.clf()

### 5. Вычислить теоретические, точечные и интервальные значения характеристик двумерной НСВ (математическое ожидание, дисперсия, корреляция)

In [16]:
# Ожидаемое и наблюдаемое математическое ожидание:

expected_mean = sp.integrate
observed_mean = np.mean(x_sample), np.mean(y_sample)

In [17]:
# Ожидаемая и наблюдаемая дисперсия:



In [18]:
# Ожидаемая и наблюдаемая корреляция:



### 6. *Проверить статистические гипотезы о соответствии полученных оценок характеристик (математическое ожидание, дисперсия, корреляция) случайной величины теоретическим

## Задание 2

Написать программу, реализующую метод формирования двумерной ДСВ. Матрицу распределения ДСВ задаете самостоятельно.

Выполнить статистическое исследование:
1. Проверить составляющие двумерной ДСВ на независимость;
2. Найти условные плотности распределения;
3. Построить гистограммы составляющих двумерной ДСВ;
4. *Построить гистограмму распределения двумерной ДСВ (3D-график);
5. Вычислить теоретические, точечные и интервальные значения характеристик двумерной ДСВ (математическое ожидание, дисперсия, корреляция);
6. *Проверить статистические гипотезы о соответствии полученных оценок характеристик (математическое ожидание, дисперсия, корреляция) случайной величины теоретическим.

In [19]:
P = np.array(
    [
        [0.15, 0.25, 0],
        [0.05, 0.35, 0],
        [0.15, 0.05, 0],
        [0.15, 0.05, 0],
    ]
)

In [20]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
#
# Assuming you have "2D" dataset like the following that you need
# to plot.
#
data_2d = [ [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
            [6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
            [11, 12, 13, 14, 15, 16, 17, 18 , 19, 20],
            [16, 17, 18, 19, 20, 21, 22, 23, 24, 25],
            [21, 22, 23, 24, 25, 26, 27, 28, 29, 30] ]
#
# Convert it into an numpy array.
#
data_array = np.array(data_2d)
#
# Create a figure for plotting the data as a 3D histogram.
#
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#
# Create an X-Y mesh of the same dimension as the 2D data. You can
# think of this as the floor of the plot.
#
x_data, y_data = np.meshgrid( np.arange(data_array.shape[1]),
                              np.arange(data_array.shape[0]) )

print(x_data)
#
# Flatten out the arrays so that they may be passed to "ax.bar3d".
# Basically, ax.bar3d expects three one-dimensional arrays:
# x_data, y_data, z_data. The following call boils down to picking
# one entry from each array and plotting a bar to from
# (x_data[i], y_data[i], 0) to (x_data[i], y_data[i], z_data[i]).
#
x_data = x_data.flatten()
y_data = y_data.flatten()
z_data = data_array.flatten()
ax.bar3d( x_data,
          y_data,
          np.zeros(len(z_data)),
          1, 1, z_data )
#
# Finally, display the plot.
#
plt.show()

<IPython.core.display.Javascript object>

[[0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]]
